In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yaml, sys, os
import pickle as pkl
import pulp, shutil, random

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


import warnings
warnings.filterwarnings('ignore')

### Solve Integer Programming with Genetic Algorithm + Wisdom of Artificial Crowds

Integer Programming:

optimize $$c^T x$$

subject to $$ Ax \leq b$$

and $$x \geq 0, \; x \in Z, \; \; x_j \in [lb, ub] \; \forall j$$

In [ ]:
# read in the parameters for genetic algorithm from inputs.yml
with open('inputs.yml', 'rb') as f:
    params = yaml.safe_load(f.read())

problem_size = params['n']
preload_data_flag = params['data_load']

### Data preperation

In [ ]:
class Initializer:
    def __init__(self, n, n_constraints=10):
        self.n = n
        self.n_constraints = n_constraints
    def create_vector_c(self):
        return np.random.randint(-10, 10, self.n)
    def create_matrix_A(self):
        return np.random.randint(-5, 10, (self.n_constraints, self.n))
    def create_vector_b(self):
        return np.random.randint(self.n*2, self.n*10, self.n_constraints)
    def save_initial_data(self, c, A, b, filename=None):
        with open(filename, 'wb') as f:
            pkl.dump({'c': c, 'A': A, 'b': b}, f)
    
def load_initial_data(filename):
    with open(filename, 'rb') as f:
        data = pkl.load(f)
    return data['c'], data['A'], data['b']

if preload_data_flag:
    c, A, b = load_initial_data(filename=f'data/linear_programming_data_{problem_size}.pkl')
else:
    num_constraints = max(int(problem_size/16), 5)
    initializer = Initializer(problem_size, num_constraints)
    c = initializer.create_vector_c()
    A = initializer.create_matrix_A()
    b = initializer.create_vector_b()
    initializer.save_initial_data(c, A, b, filename=f'data/linear_programming_data_{problem_size}.pkl')
    sys.exit("Data initialized and saved. Please set 'data_load' to True in inputs.yml to load the data next time.")

print(A.shape)
print(b.shape)
print(c.shape)

In [ ]:
def solve_integer_programming(c, A, b, method='cbc'):
    """"
    Maximize cᵀx
    subject to Ax ≤ b
    and x ≥ 0, x ∈ Z
    """
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    n_vars = len(c)
    n_constraints = len(b)
    # 1. Create a linear programming model (using PuLP)
    model = pulp.LpProblem("MatrixForm_LP", pulp.LpMaximize)
    # 2. Create optimization, integer variables 0 <= x0, x1, x2, ... <= 50
    x = pulp.LpVariable.dicts("x", range(n_vars), lowBound=0, upBound=50, cat="Integer")
    # 3. Objective: maximize cᵀx
    model += pulp.lpSum(c[j] * x[j] for j in range(n_vars))
    # 4. Constraints: A_i * x ≤ b_i
    for i in range(n_constraints):
        model += pulp.lpSum(A[i, j] * x[j] for j in range(n_vars)) <= b[i]
    # 5. Solve
    if method=='cbc':
        # Dual/Primal Simplex
        model.solve(pulp.COIN_CMD(path="/opt/homebrew/bin/cbc", msg=False))
    elif method=='glpk':
        # simplex + Interior Point
        model.solve(pulp.GLPK_CMD(msg=False))
    y = pulp.value(model.objective)
    return x, y

# Solve the integer programming:
# Maximize cᵀx
# subject to Ax ≤ b
# and x ≥ 0, x ∈ Z
x, obj_value = solve_integer_programming(c, A, b, method='cbc')
print("Objective value =", obj_value)
# prepare data
results_data = {
    "Variable": [f"x{j}" for j in range(problem_size)],
    "Value": [x[j].value() for j in range(problem_size)]
}
df = pd.DataFrame(results_data)
# df = df.sort_values("Variable").reset_index(drop=True)
df.to_pickle(f"data/optimal_solution_{problem_size}.pkl")
df[df["Value"] > 0].head(problem_size)

### Genetic Algorithm

In [ ]:
class GeneticAlgorithmWOC():
    def __init__(self, c, A, b, lb, ub):
        self.c = c
        self.A = A
        self.b = b
        with open('inputs.yml', 'rb') as f:
            params = yaml.safe_load(f.read())
        self.population_size = params['population_size']
        self.n_gen = params['num_generations']
        self.tournament_size = params['tournament_size']
        self.inv_p = params['inv_p']
        self.swap_p= params['swap_p']
        self.delta_mutate_p = params['delta_mutate_p']
        self.topsoln_frac = params['topsoln_frac']
        self.keep_children = params['keep_children']
        self.stall_patience = params['stall_patience']
        self.n_vars = len(c)
        self.n_constraints = len(b)
        self.lower_bound = lb
        self.upper_bound = ub

    @staticmethod
    def is_x_feasible(x, A, b):
        """ Quick check to see if Ax <= b (constraints) """
        return np.all(A @ np.array(x) <= b + 1e-9)
    
    def eval_fitness(self, solution):
        """
        Calculate the fitness of a solution, x: fitness = np.dot(c,x)
        Fitness is defined as the objective value if constraints are satisfied,
        otherwise a large negative penalty is applied.
        """
        # Check if solution is constrained feasible i.e., whether A*solution <= b,
        # If yes, return optimal c^T * x, else return large negative number
        if self.is_x_feasible(solution, self.A, self.b):
            return np.dot(self.c, solution)
        else:
            return -1e6
        
    ### HELPER FUNCTION, useful in ox and pmx crossover implementation and other functions
    def max_feasible_for_var(self, slack, col, lb, ub):
        """
        Inputs:
            slack  is the slack variable when having non-equality constraint, originally, s = b - Ax
            col    is a column of the constraint matrix A
            lb, ub are lower and upper bounds of each entry in solution vector x
        
        We will use this function for each x_j in the solution vector x

        Given slack with initially should be b - Ax or b[i] - SUM_k A[i,k] x[k]
        the following helps us evaluate how far can we increase x_j
        we know that constraint is: SUM_k A[i,k] x[k] <= b[i]
        currently, slack is:        s[i] = b[i] - SUM_k A[i,k] x[k]
        so increasing x[j] by delta adds A[i,j] * delta to left-hand side
        to stay feasible with constraint, we need A[i,j] * delta <= s[i]
        which means delta <= s[i] / A[i,j]
        => x[j] < min_i s[i] / A[i,j]
        we find the position 'pos' in col such that increasing x[pos] will reduce slack
        """
        pos = col > 0   # extract positions/indices in a column of A that are positive
        if np.any(pos):
            vmax_list = np.floor(slack[pos] / col[pos]).astype(int)
            vmax = int(vmax_list.min()) if vmax_list.size > 0 else ub
        else:
            # If all coefficients <= 0, increasing this variable doesn't hurt feasibility
            vmax = ub
        return min(ub, max(lb, vmax))   # choose feasible value for x[j]

    def generate_feasible_individual(self):
        """
        Greedy constructive heuristic to generate feasible solution x in [lb, ub]^n s.t. A x <= b is satisfied.
        """
        n = self.n_vars
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)
        x = np.zeros(n, dtype=int)
        slack = self.b.astype(float) - self.A @ x   # start with full slack = b - Ax
        order = np.random.permutation(n)            # random permutation from 0 to n-1

        # for each column in matrix A,
        #   we check if col is positive
        for j in order:
            col = self.A[:, j].astype(float)    # this is how much constrait i's LHS changes when increasing x[j]
            vmax = self.max_feasible_for_var(slack, col, lb, ub)
            vmin = lb
            # choose a value for x[j] within [vmin, vmax] 
            if vmax < vmin:
                # No room to increase without violating constraints; pick the safest value
                val = vmin if vmin == 0 else 0
            else:
                val = np.random.randint(vmin, vmax + 1)
            # update slack
            x[j] = int(val)
            slack = slack - col * val
        return x.tolist()

    # def create_init_popn(self):
    #     """
    #     Create an initial population of feasible solutions (A x <= b) with x integer in [lb, ub].
    #     """
    #     initial_pop = []
    #     tries_per_individual = 50
    #     for _ in range(self.population_size):
    #         x = None
    #         # Try greedy construction multiple times with different orders
    #         for __ in range(tries_per_individual):
    #             candidate = self.generate_feasible_individual()
    #             if self.is_x_feasible(candidate, self.A, self.b):
    #                 x = candidate
    #                 break
    #         initial_pop.append(x)
    #     return initial_pop
    
    def create_init_popn(self):
        initial = []
        tries_per_individual = 20
        while len(initial) < self.population_size:
            x = None
            for _ in range(tries_per_individual):
                cand = self.generate_feasible_individual()
                if self.is_x_feasible(cand, self.A, self.b):
                    x = cand; break
            if x is None:
                # fallback: random integer then repair
                randx = np.random.randint(self.lower_bound, self.upper_bound+1, size=self.n_vars).tolist()
                x = self.repair_project(randx)
            initial.append(x)
        return initial
    
    def choose_best_individuals(self, population, fitnesses):
        """
        Select the top fraction of individuals based on their fitness values.
        Top fraction is given by self.topsoln_frac.
        This function will be used in selecting the first few candidates in the next_population.
        """
        num_top = max(1, int(self.topsoln_frac * len(population)))  # num_top = number of top-candidates to be selected
        sorted_indices = np.argsort(fitnesses)[::-1]                # indices of num_top candidates that have best fitnesses
        best_indices = sorted_indices[:num_top]                     # sort the indices
        best_individuals = [population[i] for i in best_indices]    # extract best solutions (these should have largest fitness-values)
        return best_individuals
    
    def randomize_good_solution(self, population, fitnesses, k=10):
        """
        Choose a reasonably good solution out of "k" randomly selected solutions
        """
        k = self.tournament_size
        idx = np.random.choice(len(population), size=k, replace=False)
        best = max(idx, key=lambda i: fitnesses[i])
        return population[best]
    
    ################################# CROSSOVER IMPLEMENTATION #################################
    def ox(self,p1,p2):
        """ Order Crossover """
        n = self.n_vars
        assert len(p1) == n and len(p2) == n, "Parents must have length n_vars"
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)

        # Find the slice range randomly for ox cross over
        slice_start, slice_end = sorted(np.random.choice(np.arange(n), size=2, replace=False))

        def build_child(p1, p2):
            """
            p1 = parent 1, p2 = parent 2
            Note: we will take a slice from p1 i.e., p1[slice_start, slice_end] and capped with constraints feasibility
                  then fill remaining indices of child by p2, also capped with constraints feasibility
            """
            x = np.zeros(n, dtype=int)
            # first compute slack
            slack = self.b.astype(float) - self.A @ x
            # 1) copy the middle slice from 'p1' but cap by feasibility
            for k in range(slice_start, slice_end+1):
                col = self.A[:, k].astype(float)        # extract column k which has an effect on x[k]
                gene = int(p1[k])                       # extract the wanted 'gene' from p1
                # then compute the max feasible value for x[k], called vmax
                vmax = self.max_feasible_for_var(slack, col, lb, ub)
                val = max(lb, min(gene, vmax))          # choose x[k]'s value between [lb, gene]
                x[k] = val
                slack = slack - col * val               # update slack after choosing x[k]
            # 2) fill remaining positions in cyclic order using 'p2'
            for k in list(range(slice_end + 1, n)) + list(range(0, slice_start)):
                col = self.A[:, k].astype(float)        # extract column k which has an effect on x[k]
                gene = int(p2[k])                       # extract 'gene' from p2
                # then compute the max feasible value for x[k], called vmax
                # then follow the same logic as above for-loop to fill the remaining slot for our child
                vmax = self.max_feasible_for_var(slack, col, lb, ub)
                val = max(lb, min(gene, vmax))
                x[k] = val
                slack = slack - col * val
            return x.tolist()

        # return 2 children
        return build_child(p1, p2), build_child(p2, p1)
    
    def pmx(self,p1,p2):
        """ Partially Mapped Crossover """
        n = self.n_vars
        assert len(p1) == n and len(p2) == n, "Parents must have length n_vars"
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)

        # find the slice range randomly
        slice_start, slice_end = sorted(np.random.choice(np.arange(n), size=2, replace=False))
        # mapping from P2 -> P1 over the segment [slice_start, slice_end+1]:
        # seg_map_12: on slice: [slice_start, slice_end+1], we create a mapping of element in p1 to p2
        seg_map_12 = {k: (int(p1[k]), int(p2[k])) for k in range(slice_start, slice_end+1)}  # index -> (p1_val, p2_val)
        # map_p2_to_p1: value mapping (by PMX): a value from p2 maps to corresponding value in p1 in the segment
        map_p2_to_p1 = {v: u for (_, (u, v)) in seg_map_12.items()}
        # segment_vals_p1: the set of values in p1 that will be used by perform_mapping()
        segment_vals_p1 = set(int(p1[k]) for k in range(slice_start, slice_end+1))

        def perform_mapping(g):
            """
            Follow mapping P2->P1 until we get a value not in the P1 segment
            This is PMX logic: if the desired value g has conflict with segment that's been copied,
            then we need to follow the mapping chain: g <- map_p2_to_p1[g] <- ... 
            until we get a value that is not in p1 segment
            """
            seen = 0; n = self.n_vars
            while g in segment_vals_p1 and g in map_p2_to_p1:
                g = map_p2_to_p1[g]
                seen += 1
                if seen > n:  # avoid pathological loops
                    break
            return g
        
        def build_child(p1, p2):
            """
            p1 = parent1, p2 = parent2
            To create a child from p1 & p2:
            - Copy a random segment from p1 i.e., p1[slice_start, slice_end]
            - Fill the rest by doing a mapping from p2 -> p1.
            - Enforce constraints feasibility
            """
            x = np.zeros(n, dtype=int)
            slack = self.b.astype(float) - self.A @ x
            # 1) copy the middle segment from 'p1' with feasibility clamp
            for k in range(slice_start, slice_end+1):
                col = self.A[:, k].astype(float)
                gene = int(p1[k])
                vmax = self.max_feasible_for_var(slack, col, lb, ub)
                val = max(lb, min(gene, vmax))
                x[k] = val
                slack = slack - col * val
            # 2) fill outside segment positions using PMX mapping from 'p2'
            for k in list(range(0, slice_start)) + list(range(slice_end+1, n)):
                col = self.A[:, k].astype(float)
                gene = int(p2[k])
                gene = perform_mapping(gene)
                vmax = self.max_feasible_for_var(slack, col, lb, ub)
                val = max(lb, min(gene, vmax))
                x[k] = val
                slack = slack - col * val
            return x.tolist()

        # return 2 childrens
        return build_child(p1, p2), build_child(p2, p1)
    
    ###################################################################################################


    ##################################### MUTATION IMPLEMENTATION #####################################
    
    def mutate(self, parent, type='swap', p=0.15):
        if random.random() > p:
            return parent.copy()
        n = self.n_vars
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)

        # For swap:      find 2 indices that we need to perform swap on the parent solution
        # For inversion: find 2 indices that define the ranges of inversion within parent solution
        i, j = sorted(np.random.choice(n, size=2, replace=False))
        # recall: input is the 'parent' solution
        parent = parent.copy()
        if type  == 'swap':
            parent[i], parent[j] = parent[j], parent[i]
            prioritized_indices = [i,j]
        elif type== 'inversion':
            parent[i:j+1] = parent[i:j+1][::-1]
            prioritized_indices = list(range(i, j+1))

        # In the above, we have done the operations on parent when mutation = swap or inversion
        # Now, we need to ensure that the created child (from parent) satisfy the feasible constraints (A*child <= b)
        x  = np.zeros(n, dtype=int)     # initialize child
        slack = self.b.astype(float) - self.A @ x
        order, examined_indices  = [], []
        # we need to priority the swap indices first (if mutation is swap)
        # else, if mutation is inverse, then we priority indices in the inversion range: [i, i+1, ..., j-1, j]
        for idx in prioritized_indices:
            if 0 <= idx < n and idx not in examined_indices:
                order.append(int(idx)); examined_indices.append(int(idx))
        # after inserting priority indices, we populate other indices into the list order
        for j in range(n):
            if j not in examined_indices:
                order.append(j)
        # here, kinda repeat what we've done with cross-over i.e., finding maximum possible integer value
        #       for each x[j] s.t. Ax <= b is still satisfied
        for j in order:
            col = self.A[:, j].astype(float)
            gene = int(parent[j])
            vmax = self.max_feasible_for_var(slack, col, lb, ub)
            val = max(lb, min(gene, vmax))
            x[j] = val
            slack = slack - col * val
        return x.tolist()

    ###################################################################################################

    def uniform_crossover(self, p1, p2):
        """Uniform crossover for integer vectors; no feasibility enforced here."""
        n = self.n_vars
        child = np.empty(n, dtype=int)
        mask = np.random.rand(n) < 0.5
        child[mask]  = np.asarray(p1, dtype=int)[mask]
        child[~mask] = np.asarray(p2, dtype=int)[~mask]
        return child.tolist()

    def one_point_crossover(self, p1, p2):
        n = self.n_vars
        cut = np.random.randint(1, n)
        c1 = np.array(p1, dtype=int).copy()
        c2 = np.array(p2, dtype=int).copy()
        c1[cut:], c2[cut:] = c2[cut:], c1[cut:].copy()
        return c1.tolist(), c2.tolist()
    
    def delta_mutate(self, x, p=0.2, max_changes=3, step=3):
        """
        With prob p, perturb up to max_changes variables by +/- up to 'step' units.
        """
        if np.random.rand() > p:
            return x.copy()
        n = self.n_vars
        x = np.array(x, dtype=int)
        idxs = np.random.choice(n, size=np.random.randint(1, max_changes+1), replace=False)
        for j in idxs:
            delta = np.random.randint(-step, step+1)
            x[j] = int(np.clip(x[j] + delta, self.lower_bound, self.upper_bound))
        return x.tolist()
    
    def repair_project(self, x):
        """
        If infeasible, greedily reduce variables to satisfy Ax <= b while losing as little c^T x as possible.
        Heuristic: prioritize variables by (benefit loss per unit violation fix).
        """
        A = self.A.astype(float)
        b = self.b.astype(float)
        x = np.array(x, dtype=float)
        lb, ub = float(self.lower_bound), float(self.upper_bound)

        # Clip to bounds first
        x = np.clip(x, lb, ub)

        # Compute violation
        lhs = A @ x
        viol = lhs - b   # positive entries mean constraint violation
        if np.all(viol <= 1e-9):
            return x.astype(int).tolist()

        # Precompute how each variable affects constraints: only positive A_ij can reduce violation if we decrease x_j
        pos_mask = (A > 0)

        # Iterate until feasible (bounded to avoid long loops)
        for _ in range(5 * self.n_vars):
            lhs = A @ x
            viol = lhs - b
            if np.all(viol <= 1e-9):
                break

            # Aggregate total "pressure" per variable from violated constraints
            violated = viol > 1e-9
            if not np.any(violated):
                break

            # For each j, how much does decreasing x_j help? score_j = sum_i A_ij over violated i
            help_j = np.sum(np.where(violated[:, None], np.where(pos_mask, A, 0.0), 0.0), axis=0)

            # Ratio: loss in objective per unit of violation relief (prefer small)
            # loss per unit decrease in x_j is c_j; relief per unit decrease approx help_j
            c = self.c.astype(float)
            with np.errstate(divide='ignore', invalid='ignore'):
                ratio = np.where(help_j > 1e-12, c / help_j, np.inf)

            j = np.argmin(ratio)  # best variable to decrease

            if not np.isfinite(ratio[j]) or help_j[j] <= 1e-12:
                # Fallback: decrease a random variable that can help
                candidates = np.where(help_j > 1e-12)[0]
                if candidates.size == 0:
                    break
                j = np.random.choice(candidates)

            # Compute max decrease for x_j before either feasibility or lb hits
            # We want to reduce violation to <= 0: for each violated i, decrease Δ so A_ij * Δ >= viol_i
            col = A[:, j]
            pos_rows = (col > 0) & violated
            if np.any(pos_rows):
                # minimum Δ to fix all violated constraints along this column
                delta_needed = np.max(viol[pos_rows] / col[pos_rows])
            else:
                delta_needed = 0.0

            # Decrease at least 1 unit, but not below lb
            new_val = max(lb, x[j] - max(1.0, np.ceil(delta_needed)))
            if new_val == x[j]:
                # can't move this var; zero-out its help to avoid infinite loop
                help_j[j] = 0.0
                continue

            x[j] = new_val

        # Final clip and round
        x = np.clip(x, lb, ub)
        return np.rint(x).astype(int).tolist()



    
    def genetic_alg_main(self, seed=42):
        """
        Main loop for the genetic algorithm:
        - GA:
            - OX and PMX crossovers
            - swap and inversion mutation
        - WOC:
        """
        if seed is not None:
            random.seed(seed)
        # 1. Create initial population
        population = self.create_init_popn()
        best_fitness_over_time = []
        fitness = [self.eval_fitness(ind) for ind in population]
        best_fitness = max(fitness)
        best_solution = population[fitness.index(best_fitness)]

        stall = 0
        hist_best, hist_mean = [], []
        for gen in range(self.n_gen):
            curr_best_fitness = max(fitness)
            curr_mean = np.mean(fitness)
            hist_best.append(curr_best_fitness)
            hist_mean.append(curr_mean)
            if curr_best_fitness > best_fitness:
                best_fitness = curr_best_fitness
                best_solution= population[fitness.index(best_fitness)]
                stall = 0
            else:
                stall += 1
                if stall >= self.stall_patience: break

            # find next population
            next_population = self.choose_best_individuals(population, fitness)
            while len(next_population) < self.population_size:
                # select 2 parents using randomize_good_solution
                p1 = self.randomize_good_solution(population, fitness)
                p2 = self.randomize_good_solution(population, fitness)
                while p1 == p2: # rarely happen
                    p2 = self.randomize_good_solution(population, fitness)
                # # perform crossovers (OX & PMX) to generate 4 childrens
                # ox1, ox2 = self.ox(p1,p2)
                # pmx1,pmx2= self.pmx(p1,p2)
                # # perform mutations (SWAP & INVERSION) on the 4 childrens
                # children = [
                #     # do swap and inversion mutations for each of ox1, ox2, pmx1, pmx2
                #     self.mutate(ox1, type='swap',      p=self.swap_p),
                #     self.mutate(ox1, type='inversion', p=self.inv_p),
                #     self.mutate(ox2, type='swap',      p=self.swap_p),
                #     self.mutate(ox2, type='inversion', p=self.inv_p),
                #     self.mutate(pmx1, type='swap',     p=self.swap_p),
                #     self.mutate(pmx1, type='inversion',p=self.inv_p),
                #     self.mutate(pmx2, type='swap',     p=self.swap_p),
                #     self.mutate(pmx2, type='inversion',p=self.inv_p)
                # ]

                # produce 2 children

                c1, c2 = self.one_point_crossover(p1, p2)
                c3 = self.uniform_crossover(p1, p2)
                c4 = self.uniform_crossover(p2, p1)
                children = [c1, c2, c3, c4]
                children_updated = []
                # mutate by value
                for ch in children:
                    ch = self.delta_mutate(ch, p=self.delta_mutate_p, max_changes=4, step=4)
                    ch = self.repair_project(ch)
                    children_updated.append(ch)
                # update children after mutation and repair
                children = [ch for ch in children_updated]

                # evaluate fitness of each child
                children_fitness = [self.eval_fitness(ch) for ch in children]
                # push best-needed children into next_population
                best_children_indices = sorted(
                                range(len(children)),
                               key=lambda i: children_fitness[i],
                               reverse=True)[:self.keep_children]
                for i in best_children_indices:
                    next_population.append(children[i])

            ##################################################################################################
            ################################ Wisdom of Artificial Crowds #####################################
            ##################################################################################################
            # Implementation may go here


            ##################################################################################################


            # update population and go to next GA-generation:
            population = next_population
            fitness = [self.eval_fitness(w) for w in population]

        #---------------- End of for gen in ... loop -----------------#
        opt_soln = best_solution
        opt_obj_val = self.eval_fitness(opt_soln)
        out = {
            'optimal_solution' : opt_soln,
            'optimal_objective': opt_obj_val,
            'history_of_fitness': hist_mean
        }
        return out

In [ ]:
list_of_seeds = random.sample(range(5*params['n_ga_runs']), params['n_ga_runs'])
print(list_of_seeds)

for sd in list_of_seeds:
    ga = GeneticAlgorithmWOC(c, A, b, lb=0, ub=50)
    results = ga.genetic_alg_main(seed=sd)
    print(results['optimal_objective'])


In [ ]:
df_ga = pd.DataFrame({
    "Variable": [f"x{j}" for j in range(problem_size)],
    "Value": results['optimal_solution']
})
df_ga[df_ga["Value"] > 0].head(problem_size)

In [ ]:
df[df["Value"] > 0].head(problem_size)